In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2 
from sklearn.feature_extraction.text import TfidfVectorizer
from math import sqrt
import pickle
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
ayam = pd.read_csv('dataset-ayam.csv')
ikan = pd.read_csv('dataset-ikan.csv')
kambing = pd.read_csv('dataset-kambing.csv')
sapi = pd.read_csv('dataset-sapi.csv')
tahu = pd.read_csv('dataset-tahu.csv')
telur = pd.read_csv('dataset-telur.csv')
tempe = pd.read_csv('dataset-tempe.csv')
udang = pd.read_csv('dataset-udang.csv')

In [3]:
df = ayam[:13].append([ikan[:13],kambing[:13],sapi[:13],tahu[:13],telur[:13],tempe[:13],udang[:13]])

C:\Users\Aryasatya\AppData\Local\Temp\ipykernel_22148\3628990811.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = ayam[:13].append([ikan[:13],kambing[:13],sapi[:13],tahu[:13],telur[:13],tempe[:13],udang[:13]])


In [4]:
df = df.dropna()

In [5]:
a = df.isnull().sum()
print(a)

Title          0
Ingredients    0
Steps          0
Loves          0
URL            0
dtype: int64


In [6]:
df = df.drop(['Loves', 'URL'],axis='columns')

In [7]:
def removeWords(text):   
    text = text.lower()                      
    text = re.sub(r'[^\w\s]',"  ", text)     
    text = text.strip()
    text = re.sub(r'[-+]?[0-9]+', '', text)   
    return text

In [8]:
def case_folding(text):
    text = text.lower()
    return text

In [9]:
key_norm = pd.read_csv('key_norm_merge.csv')

In [10]:
def text_normalize(text):
    text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])
    text = str.lower(text)
    return text

In [11]:
stopwords_ind = stopwords.words('indonesian')

In [12]:
more_stopword = ['kg', 'secukupnya', 'selera']                   

stopwords_ind = stopwords_ind + more_stopword

def remove_stop_words(text):
    clean_words = []
    text = text.split()
    for word in text:
        if word not in stopwords_ind:
            clean_words.append(word)
    return ' '.join(clean_words)

In [13]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    text = stemmer.stem(text)
    return text

In [14]:
raw_sample = df['Ingredients'].iloc[5]
remove_words = removeWords(raw_sample)
stopword_removal = remove_stop_words(remove_words)
text_stemming = stemming(stopword_removal)

print('Raw data\t\t: ', raw_sample)
print('Remove Words\t\t: ', remove_words)
print('Stopword removal\t: ', stopword_removal)
print('Stemming\t\t: ', text_stemming)

Raw data		:  1 piring nasi--¹/⁴ fillet ayam, potong kotak, cuci bersih--Daun pisang yang sudah dibersihkan--Lidi/tusuk gigi untuk mengapit daun--1 lbr daun salam--3 cm serai, memarkan--1 ikat daun kemangi, opsional--1 cabe merah besar, iris tipis--1 sdt kecap manis--Minyak untuk menumis--100 ml air--Bumbu Halus:--1 cabe merah besar--3 cabe kecil--2 siung bawang merah--1 siung bawang putih--1 kemiri--1 cm kunyit--¹/² ruas jahe--¹/⁴ tomat--1 sdt gula pasir--1 blok kaldu ayam (aku biasa pake maggy) pake royco, masako--Merica (dikira2 z takar nya, aku sdkit z sih)--
Remove Words		:   piring nasi    ¹  ⁴ fillet ayam   potong kotak   cuci bersih    daun pisang yang sudah dibersihkan    lidi  tusuk gigi untuk mengapit daun     lbr daun salam     cm serai   memarkan     ikat daun kemangi   opsional     cabe merah besar   iris tipis     sdt kecap manis    minyak untuk menumis     ml air    bumbu halus       cabe merah besar     cabe kecil     siung bawang merah     siung bawang putih     kemiri

In [15]:
def text_preprocessing_process(text):
    text = removeWords(text)
    text = text_normalize(text)
    text = remove_stop_words(text)
    text = stemming(text)
    return text

In [16]:
%%time
df['clean_teks'] = df['Ingredients'].apply(text_preprocessing_process)

CPU times: total: 13 s
Wall time: 13 s


In [17]:
df.to_csv('clean_data.csv')

In [18]:
X = df['clean_teks']
y = df['Ingredients']

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_X = TfidfVectorizer(ngram_range=(1,1))
tf_idf_X.fit(X)

TfidfVectorizer()

In [20]:
X_tf_idf = tf_idf_X.transform(X).toarray()
X_tf_idf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.21879812,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [21]:
data_tf_idf_X = pd.DataFrame(X_tf_idf, columns=tf_idf_X.get_feature_names_out())
data_tf_idf_X

,abc,acar,adas,adon,air,al,ambil,amis,an,angciu,...,utama,utuh,vaname,vetsin,wajib,wedges,wijen,wortel,ya,zaitun
0,0.0,0.0,0.0,0.0,0.117864,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.091114,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.218798,0.0
2,0.0,0.0,0.0,0.0,0.176830,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.0,0.0,0.0,0.0,0.117644,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.328494,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
99,0.0,0.0,0.0,0.0,0.113208,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
100,0.0,0.0,0.0,0.0,0.115222,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
101,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [22]:
with open('tf_idf_feature.pickle', 'wb') as output:
    pickle.dump(X_tf_idf, output)

In [23]:
tf_idf_y = TfidfVectorizer(ngram_range=(1,1))
tf_idf_y.fit(y)

TfidfVectorizer()

In [24]:
y_tf_idf = tf_idf_y.transform(y).toarray()
y_tf_idf

array([[0.30065928, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.13305151, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.2091175 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [25]:
data_tf_idf_y = pd.DataFrame(y_tf_idf, columns=tf_idf_y.get_feature_names_out())
data_tf_idf_y

,10,100,1000,12,13,15,150,180,1kg,20,...,vaname,vetsin,wajib,wedges,wijen,wortel,ya,yang,yg,zaitun
0,0.300659,0.000000,0.0,0.23756,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.16285,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
3,0.000000,0.167403,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.293614,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
99,0.147020,0.000000,0.0,0.00000,0.0,0.0,0.23233,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
100,0.133052,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
101,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0


In [26]:
X = np.array(data_tf_idf_X)
# y = np.array(data_tf_idf_y)
y = np.array(y)

In [27]:
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2 

# Ten features with highest chi-squared statistics are selected 
chi2_features = SelectKBest(chi2, k='all') 
X_kbest_features = chi2_features.fit_transform(X, y) 
  
# Reduced features 
print('Original feature number:', X.shape[1]) 
print('Reduced feature number:', X_kbest_features.shape[1]) 

Original feature number: 479
Reduced feature number: 479


In [28]:
data_chi2 = pd.DataFrame(chi2_features.scores_, columns=['nilai'])
feature = tf_idf_X.get_feature_names_out()
data_chi2['fitur'] = feature
mask = chi2_features.get_support()
mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [29]:
new_feature = []

for bool, f in zip(mask, feature):
    if bool:
        new_feature.append(f)
    selected_feature = new_feature

selected_feature

['abc',
 'acar',
 'adas',
 'adon',
 'air',
 'al',
 'ambil',
 'amis',
 'an',
 'angciu',
 'anis',
 'anise',
 'api',
 'apit',
 'arah',
 'asa',
 'asam',
 'asin',
 'asli',
 'avocado',
 'awang',
 'ayak',
 'ayam',
 'baba',
 'babat',
 'babom',
 'bag',
 'bahan',
 'bakar',
 'baking',
 'bakso',
 'bakwan',
 'balur',
 'bamer',
 'bango',
 'banyak',
 'baput',
 'barbeque',
 'basah',
 'basil',
 'batagor',
 'batang',
 'batu',
 'bau',
 'bawang',
 'bawng',
 'bb',
 'bebek',
 'beef',
 'beli',
 'belimbing',
 'beras',
 'bersih',
 'bh',
 'biar',
 'bihun',
 'biji',
 'black',
 'blender',
 'blok',
 'bngkus',
 'bombai',
 'bombang',
 'bombay',
 'bonggol',
 'bongkol',
 'botol',
 'bread',
 'broiler',
 'brokoli',
 'brown',
 'btng',
 'btr',
 'buah',
 'buang',
 'bubuk',
 'bulat',
 'bumbu',
 'buncir',
 'buncis',
 'bunga',
 'bungkul',
 'bungkus',
 'butir',
 'bwg',
 'bwng',
 'cabai',
 'cacah',
 'cage',
 'cair',
 'campur',
 'cc',
 'ceker',
 'cemplung',
 'cengek',
 'cengjeh',
 'cengkeh',
 'cepat',
 'cheddar',
 'cheedar',
 'c

In [30]:
kbest_feature = {} 

for (k,v) in tf_idf_X.vocabulary_.items():    
    if k in selected_feature:                 
        kbest_feature[k] = v  
        
kbest_feature

{'ekor': 119,
 'ayam': 22,
 'kampung': 184,
 'potong': 339,
 'buah': 73,
 'jeruk': 174,
 'nipis': 302,
 'sdm': 380,
 'garam': 128,
 'ruas': 357,
 'kunyit': 233,
 'bawang': 44,
 'merah': 284,
 'putih': 346,
 'cabai': 86,
 'rawit': 350,
 'sesuai': 402,
 'butir': 83,
 'kemiri': 203,
 'batang': 41,
 'sereh': 399,
 'lembar': 249,
 'daun': 111,
 'salam': 365,
 'ikat': 157,
 'kemangi': 199,
 'sedap': 388,
 'gelas': 131,
 'air': 4,
 'ya': 477,
 'serai': 397,
 'memar': 281,
 'halus': 145,
 'ketumbar': 211,
 'jari': 169,
 'laos': 240,
 'tuk': 455,
 'ukep': 463,
 'minyak': 289,
 'goreng': 138,
 'hijau': 149,
 'siung': 408,
 'gula': 141,
 'tomat': 452,
 'gr': 139,
 'daging': 110,
 'pakai': 310,
 'fillet': 124,
 'tepung': 441,
 'serbaguna': 398,
 'lalap': 239,
 'kol': 217,
 'timun': 445,
 'panas': 313,
 'sambal': 366,
 'korek': 218,
 'bwg': 84,
 'kulit': 228,
 'lemak': 248,
 'kating': 193,
 'cincang': 102,
 'kasar': 192,
 'jahe': 166,
 'geprek': 134,
 'ml': 291,
 'bubuk': 75,
 'piring': 336,
 'nasi

In [31]:
data_selected_feature = pd.DataFrame(X_kbest_features, columns=selected_feature)
data_selected_feature

,abc,acar,adas,adon,air,al,ambil,amis,an,angciu,...,utama,utuh,vaname,vetsin,wajib,wedges,wijen,wortel,ya,zaitun
0,0.0,0.0,0.0,0.0,0.117864,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.091114,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.218798,0.0
2,0.0,0.0,0.0,0.0,0.176830,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.0,0.0,0.0,0.0,0.117644,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.328494,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
99,0.0,0.0,0.0,0.0,0.113208,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
100,0.0,0.0,0.0,0.0,0.115222,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
101,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [32]:
data_selected_feature.to_csv('data_selected_feature.csv')

In [33]:
with open('kbest_feature.pickle', 'wb') as output:
    pickle.dump(kbest_feature, output)

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [35]:
y = np.array(data_tf_idf_y)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.4)

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier(n_neighbors = 8)
# classifier.fit(X_train, y_train)

In [39]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

multi_output_clf = MultiOutputRegressor(Ridge(random_state=123))
multi_output_clf.fit(X_train,y_train)

MultiOutputRegressor(estimator=Ridge(random_state=123))

In [40]:
# y_pred = classifier.predict(X_test)

In [41]:
y_pred = multi_output_clf.predict(X_test)

In [42]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute
from numpy import mean
from numpy import std

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [43]:
n_scores = cross_val_score(multi_output_clf, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

n_scores = absolute(n_scores)

print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 0.011 (0.001)


In [44]:
from sklearn.datasets import make_regression
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR

X_train, y_train = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)

model = LinearSVR()

row = X_train

print('Predicted: %s' % row)

Predicted: [[ 2.19477494e-01  3.29489967e-01  8.15600360e-01 ... -2.90545028e-03
   9.64022632e-01  4.99224881e-02]
 [ 8.43368646e-01  5.64675313e-01  3.28040418e-01 ...  5.53385034e-02
   1.36536046e-01  6.30310630e-01]
 [-1.04499819e+00 -8.83771319e-01  7.67113096e-01 ...  4.82597952e-01
   4.20755200e-01  1.63888893e+00]
 ...
 [ 1.05533584e+00  6.93456049e-01 -4.02886049e-02 ...  1.77556598e+00
   7.86215852e-01  1.40686786e+00]
 [-1.21333813e+00 -3.79517516e-01  4.35986196e-01 ...  9.66220863e-01
   3.32107876e+00 -2.69836174e+00]
 [-8.13514715e-01 -1.02528191e+00  7.01965245e-01 ... -2.74046516e-01
   8.56770375e-01  1.86000596e+00]]


In [45]:
from joblib import dump   
model = multi_output_clf.fit(X_train, y_train)   

dump(model, filename='model_1.joblib')

['model_1.joblib']

In [46]:
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# result = confusion_matrix(y_test, y_pred)
# print("Confusion Matrix:")
# print(result)
# result1 = classification_report(y_test, y_pred)
# print("Classification Report:",)
# print (result1)
# result2 = accuracy_score(y_test,y_pred)
# print("Accuracy:",result2)

In [47]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

In [48]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.4)
# from sklearn.neighbors import KNeighborsRegressor
# knnr = KNeighborsRegressor(n_neighbors = 10)
# knnr.fit(X_train, y_train)
# y_pred = knnr.predict(X_test)

In [49]:
# mse = np.power(y-knnr.predict(X),2).mean()
# print ("The MSE is:",mse)
# import math
# rmse = math.sqrt(mse)
# print("The RMSE is:", rmse)